In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

img = cv.imread('Crop_field_cropped.jpg', cv.IMREAD_GRAYSCALE)


edges = cv.Canny(img, 550 , 690)
indices = np.where(edges != [0])
x = indices[1]
y = indices[0]


plt.scatter(x, y)
plt.title('Scatter Plot')
plt.show()

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

img = cv.imread('Crop_field_cropped.jpg', cv.IMREAD_GRAYSCALE)

edges = cv.Canny(img, 550, 690)
indices = np.where(edges != [0])
x = indices[1]
y = indices[0]

m,c = np.polyfit(x, y, 1)
y_ = m*x + c

# Construct the data matrix
B = np.vstack([x, y]).T

# Calculate the centroid of the data points
centroid = np.mean(B, axis=0)

# Center the data points around the centroid
centered_data = B - centroid

# Perform Singular Value Decomposition (SVD) on the centered data
U, S, Vt = np.linalg.svd(centered_data, full_matrices=False)

# Compute the slope and intercept for the total least squares fit line
m_t = -Vt[0, 0] / Vt[1, 0]
c_t = centroid[1] - m_t * centroid[0]

# Calculate the y values for the total least squares fit line
y_t = m_t * x + c_t

# Initialize RANSAC model
ransac = RANSACRegressor()

# Fit RANSAC model
ransac.fit(x.reshape(-1, 1), y)

# Extract slope and intercept
m_ransac = ransac.estimator_.coef_[0]
c_ransac = ransac.estimator_.intercept_

# Calculate y values for the RANSAC fit line
y_ransac = m_ransac * x + c_ransac



plt.scatter(x, y, s=2, label='Noisy points')
plt.title('Scatter Plot  least-squares-fitted lines')
plt.plot(x, y_, color='r', linewidth=1, label=r'least-squares-fit line')
plt.plot(x, y_t, c='g', label='Fitted Line (TLS)')
plt.plot(x, y_ransac, color='b', label='RANSAC Fit Line')
plt.title('RANSAC Fit Line')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.show()

In [ ]:
from sklearn.linear_model import RANSACRegressor
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

img = cv.imread('Crop_field_cropped.jpg', cv.IMREAD_GRAYSCALE)


edges = cv.Canny(img, 550 , 690)
indices = np.where(edges != [0])
x = indices[1]
y = indices[0]

# Initialize RANSAC model
ransac = RANSACRegressor()

# Fit RANSAC model
ransac.fit(x.reshape(-1, 1), y)

# Extract slope of the fitted line
slope = ransac.estimator_.coef_[0]

# Calculate angle from slope (assuming y-axis is pointing downwards)
angle_deg = np.degrees(np.arctan(slope))

print("Estimated angle of the crop field: {:.2f} degrees".format(angle_deg))

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import numpy as np
import matplotlib.pyplot as plt


train_dir = 'Train'
validation_dir = 'Test'

batch_size = 1
img_size = (64, 64)

#Training Dataset
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    image_size=img_size,
    seed=42
)

#Validation Dataset
validation_dataset = tf.keras.utils.image_dataset_from_directory(
  validation_dir,
  seed=123,
  image_size=img_size,
  batch_size=batch_size
  )

class_names = train_dataset.class_names

model = models.Sequential()
model.add(layers.Conv2D(64, (5,5), activation='relu', input_shape=(64,64,3)))
model.add(layers.AveragePooling2D(2,2))
model.add(layers.Conv2D(32, (3,3), activation='relu'))
model.add(layers.AveragePooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(104, activation='relu'))
model.add(layers.Dense(76, activation='relu'))
model.add(layers.Dense(38))

model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
print(model.summary())
Fitting = model.fit(train_dataset, epochs=5, validation_data=validation_dataset)

test_loss, test_accuracy = model.evaluate(validation_dataset, verbose=2)
print('Test accuracy = ', test_accuracy)
